# TP 1 EX 1
Grupo 5:
     
     Breno Fernando Guerra Marrão A97768
     
     Tales André Rovaris Machado A96314

## Inicialização
Usamos as bibliotecas pysmt e numpy para resovler o problema de alocação proposto 

In [1]:
from pysmt.shortcuts import *
from pysmt.typing import BOOL
import numpy


## Implementação

A seguinte função cria a variável x de acordo com a probabilidade p da borda e o tamanho n

In [ ]:
def init(n,p):
    x = {}
    for i in range(n):
        x[i,0] = numpy.random.choice([TRUE(),FALSE()], p=[p,1-p])
    for j in range(n):
        x[0,j] = numpy.random.choice([TRUE(),FALSE()], p=[p,1-p])
    for i in range(1,n):
        for j in range(1,n):
            x[i,j] = FALSE()
    centro = (numpy.random.choice(numpy.arange(2, n-1)),numpy.random.choice(numpy.arange(2, n-1)))
    for i in range(-1,2):
        for j in range(-1,2):
            a = centro[0]+i
            b = centro[0]+j
            x[a,b] = TRUE()
    
    return x

In [ ]:
Função que retorna o número de vizinhos vivos em uma certa posição (a,b) na matriz curr de tamanho n.

In [ ]:
def checavizinhos(curr,a,b,n):
    if a == n-1:
        arr1 = [-1,0]
    else:
        arr1 = [-1,0,1]
    if b == n-1:
        arr2 = [-1,0]
    else:
        arr2 = [-1,0,1]
    c = 0
    for i in arr1:
        for j in arr2:
            if curr[a+i,b+j]==TRUE():
                c += 1
    if curr[a,b]==TRUE():
        c -= 1
    return c

Função de transição da matriz curr para uma nova matriz fazendo a transição de cada celula .

In [ ]:
def trans(curr,n):
    currnew = {}
    for i in range(n):
        currnew[i,0] = curr[i,0]
    for j in range(n):
        currnew[0,j] = curr[0,j]
    for i in range(1,n):
        for j in range(1,n):
            c = checavizinhos(curr,i,j,n)
            if curr[i,j]==TRUE():
                if c not in [2,3]:
                    currnew[i,j] = FALSE()
                else:
                    currnew[i,j] = TRUE()
            else:
                if c == 3:
                    currnew[i,j] = TRUE()
                else:
                    currnew[i,j] = FALSE()
    return currnew

Função que imprime a matriz.

In [ ]:
def printM(x,n):
    for i in range(n):
        for j in range(n):
            if (x[i,j]==TRUE()):
                print("X" , end = "|")
            else:
                print(".", end = "|")
        print()

Função que roda a transição k vezes

In [ ]:
def roda(n,p,k,trans):
    x = init(n,p)
    printM(x,n)
    for i in range(k):
        x = trans(x,n)
        printM(x,n)
        print("---------")

## Invariantes

Toda a célula normal está viva pelo menos uma vez em algum estado acessível
$$\forall_{1 < a< n}  \cdot \forall_{1 < b< n}\quad \sum_{x \in lista} x_{a,b} \ge 1$$

In [7]:
def vivaumavez(lista,n,a,b):
    for x in lista:
        if x[a,b]==TRUE():
            return TRUE()
    return FALSE()

Todos os estados acessíveis contém pelo menos uma célula viva
$$\sum_{1 < a< n,1 < b< n} x_{a,b} \ge 1$$

In [ ]:
def vivaumavez(x,n):
    for a in range(1,n):
        for b in range(1,n):
            if x[a,b]==TRUE():
                return TRUE()
    return FALSE()

Funções que testam os invariantes


In [ ]:

def testinva(n,p,k,trans,inv):
    
    with Solver(name="z3") as s:
        
        lista  = []
        x = init(n,p)
            
        for i in range(k):
            x = trans(x,n)
            s.add_assertion(Not(inv(x,n)))
            
        if s.solve():
            print("invariante não é valido")
            return
        print("invariante valido")

def testinvb(n,p,k,trans,inv):
    
    with Solver(name="z3") as s:
        
        lista  = []
        x = init(n,p)
        lista.append(x)        
            
        for i in range(k):
            x = trans(x,n)
            lista.append(x)
            
       
        
        s.add_assertion(Not(And([inv(lista,n,a,b) for a in range(n) for b in range(n)])))
            
        if s.solve():
            print("invariante não é valido")
            return
        print("invariante valido")
        

testinvb(10,0.5,10,trans,vivaumavez) 